# DU IA et Santé - Atelier NLP

Dans cet atelier, nous allons voir plusieurs techniques de NLP "moderne", principalement basées sur les transformers et la librairie [huggingface](TODO).

Cet atelier est diviser en trois partie.

Dans la partie 1, nous allons nous intéresser à la partie encodage des transformers (BERT et cie.) afin de mieux comprendre les représentations internes de ceux-ci.

Dans la partie 2, nous interesserons à la partie decodage des transformers (GPT et cie.) et aux méthodes de *prompt engineering* permettant d’améliorer leurs résultats.

Enfin, dans la partie 3, nous allons voir comment méler les deux approches pour construire des assistants personnels.

Mais, avant toute choses, nous allons importer quelques librairies et vérifier que nous accédons bien à voir GPU (si vous en avez un).

Pour cela, exécutez la cellule ci-dessous.

In [3]:
# Librairies utilitaires
from tqdm.auto import tqdm

# Librairies ML
import torch
print("cuda available?", str(torch.cuda.is_available()))
device = torch.device("cpu")
if torch.cuda.is_available():
  device = torch.device("cuda:0")
  torch.cuda.set_device(device)
  print("cuda version:", torch.version.cuda)
  print("cuDNN enabled?", torch.backends.cudnn.enabled)
  print("cuDNN version:", torch.backends.cudnn.version())
  print("Device name? ", torch.cuda.get_device_name(torch.cuda.current_device()))

cuda available? False


Si le résultat est : `cuda available? False`, cela signifie que la librairie *torch* n’a pas pu accéder à votre GPU.

Dans le cas contraire (ou si vous n’avez pas de GPU à disposition), nous pouvons passer à la suite.

N.B.: si vous utiliser ce GoogleCollab, voir TODO

## Partie 1 - Encodage et représentations internes

Dans cette partie, nous allons voir comment les transformers, plus spécifiquement les modèles de type BERT, encodent et transforment les textes fournis pour les donner en entrée des modèles. Nous allons aussi étudier les représentations internes de ce modèles, leurs espaces latents, et comment manipuler ces réprésentations.

Pour cela, nous allons nous baser sur le modèle [CamemBERT](TODO), entrainé sur des textes en français.

N.B.: pour essayer un autre modèle, voir: TODO

In [4]:
from transformers import CamembertModel, CamembertTokenizer

model_name = "camembert/camembert-base"

tokenizer = CamembertTokenizer.from_pretrained(model_name)
model = CamembertModel.from_pretrained(model_name)

model.to(device) # charge le modele sur le CPU/GPU

CamembertModel(
  (embeddings): CamembertEmbeddings(
    (word_embeddings): Embedding(32005, 768, padding_idx=0)
    (position_embeddings): Embedding(514, 768, padding_idx=0)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): CamembertEncoder(
    (layer): ModuleList(
      (0-11): 12 x CamembertLayer(
        (attention): CamembertAttention(
          (self): CamembertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): CamembertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
     

### 1.1 Tokenization et encodage

La première étape pour permettre à un modèle de language de traiter du texte est la tokenisation.

Cette étape permet de découper une phrase ou un mot en un ou plusieurs *token* connus du modèle (l’ensemble des tokens connus par un modèle est appelé son "vocabulaire").

Par exemple:

In [10]:
sentence = "fromage"
tokenized_sentence = tokenizer.tokenize(sentence)
tokenized_sentence

['▁fromage']

Cependant, un modèle attend en entrée non pas une liste de tokens, mais la liste des id correspondant à ces tokens dans son vocabulaire.

Ainsi, il nous faut encoder les tokens obtenus précédemment en une liste d’entier utilisable par le modèle.

In [6]:
encoded_sentence = tokenizer.encode(tokenized_sentence, return_tensors="pt")
encoded_sentence

tensor([[   5, 5271,    6]])

Le résultat ci-dessus présente la liste des ids correspondant aux tokens de notre phrase.

Nous pouvons cependant noter la précense de deux ids supplémentaires (5 et 6), ceux-ci correspondants aux tokens de début et fin de phrase.

Sentez-vous libre de modifier la phrase d’exemple avant de passer à la suite.

### 1.2 Word Embeddings et espaces latents

Maintenant que nous avons comment tokeniser et encoder un texte pour permettre à un modèle de manipuler ce texte, nous allons voir les réprentations internes utilisé par ces modèles pour manipuler ces textes.

Pour cela nous allons utiliser la classe *pipeline* de la librairie *transformers*.

In [7]:
from transformers import pipeline

Cette classe permet d’utiliser nos modèles sur divers problèmes.

Ici, nous allons l’utiliser pour effectuer de la *feature-extraction* et ainsi obtenir les représentations interne des phrases fournies à notre modèle.

In [9]:
pipeline = pipeline('feature-extraction', model=model, tokenizer=tokenizer, device=device)

Par exemple:

In [11]:
data = pipeline(sentence)
print(data)

[[[-0.05918966978788376, 0.15585827827453613, 0.15874092280864716, -0.1547703891992569, -0.07191170006990433, 0.00942622497677803, 0.05405541509389877, 0.1471589207649231, -0.0049822330474853516, 0.027379479259252548, -0.05600949749350548, 0.1741418093442917, -0.10524965822696686, 0.1021953821182251, 0.22133994102478027, -0.0007261000573635101, 0.02719491720199585, -0.0511489175260067, 0.09355249255895615, -0.15209782123565674, 0.04878760874271393, -0.07677026093006134, -0.04878748953342438, -0.3043932318687439, 0.2269667685031891, -0.18198101222515106, -0.005425736308097839, -0.18855461478233337, 0.029949355870485306, 0.06337694823741913, -0.011845819652080536, -0.20610778033733368, 0.04792089760303497, 0.13431188464164734, 0.12819388508796692, -0.12255488336086273, -0.0557720921933651, 0.06597660481929779, -0.11006240546703339, -0.17551110684871674, -0.015516575425863266, 0.11157923936843872, 0.2489694356918335, -0.08189058303833008, 0.06605683267116547, 0.2636236846446991, -0.243579

Nous voyons ici, que notre modèle transforme chaque token de notre phrase en vecteurs de taille:

In [16]:
len(data[0][1])

768

La taille de ces vecteurs, générallement appellés *embeddings*, determine le nombre de dimensions de l’espace latent de notre modèle.

Chaque vecteur correspondant alors à un point dans cet espace.

In [ ]:
embedded_vocab = pipeline(list(tokenizer.get_vocab().keys()), batch_size=32)